# Union find
#### 找有多少个group时，防止出现 0:1, 1:2, 2:2这样的局面，常常最后还需要再遍历find一次
547. Friend Circles,找有多少个group，对每个点添加到其根节点下
305. Number of Islands II, m*n的grid上不断添加节点，更新当前岛屿个数
684. Redundant Connection, 一个树里多了一条边，以至于得到了无向图，问多余的边
721. Accounts Merge，给定名字和账户，返回一个名字与其对应的所有账户
323. Number of Connected Components in an Undirected Graph，一堆edges判断有多少个group
261. Graph Valid Tree,给定一堆edges，判断是否只有一棵无环树
1202. Smallest String With Swaps，字符串s，pairs中给定的index可以任意调换，求最小字符串
1101. The Earliest Moment When Everyone Become Friends, 用groups来判断圈子的个数
765. Couples Holding Hands，交换人，使得couple相邻
1258. Synonymous Sentences,同义词替换，按照顺序输出结果
959. Regions Cut By Slashes,将每个格子分成4份，代表着不同形态的划分能产生的，可能划分方式，然后对不同小格子之间做联合
947. Most Stones Removed with Same Row or Column,只要同行或者同列存在石头，就可以删除，问最多删除多少个
924. Minimize Malware Spread,病毒蔓延，去掉哪个病毒影响最大
928. Minimize Malware Spread II
952. Largest Component Size by Common Factor，一组数分成几个group，内部互不互质，最大group size

In [ ]:
class UF(object):
    def __init__(self, size):
        self.parent = []
        self.rank = []
        self.count = size
        for i in range(0, size):
            self.parent.append(i)
            self.rank.append(0)
 
    def find(self, i):#返回i的根节点，也可以更新i父节点的根节点
        # with path compression
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]
 
    def connected(self, i, j):
        return self.find(i) == self.find(j)
 
    def union(self, i, j):#将i,j合并
        p = self.find(i)
        q = self.find(j)
        if p == q: return
        if self.rank[p] < self.rank[q]:#q rank更大
            self.parent[p] = q
        elif self.rank[p] > self.rank[q]:
            self.parent[q] = p
        else:
            self.parent[q] = p#p成了root，rank+1
            self.rank[p] += 1
        self.count -= 1

In [243]:
# 547. Friend Circles,找有多少个group，对每个点添加到其根节点下
def findCircleNum(M):
    """
    :type M: List[List[int]]
    :rtype: int
    """
    def find(node):#返回根节点,也可以更新node父节点的根节点
        if circles[node] == node: return node
        circles[node] = find(circles[node])
        return circles[node]

    n = len(M)
    circles = {x:x for x in range(n)}
    num = n
    for i in range(n):
        for j in range(i+1, n):
            if M[i][j] == 1 and find(i) != find(j):
                circles[circles[j]] = find(i)
                print(circles)
    return sum([1 for k, v in circles.items() if k == v])
findCircleNum([[1,0,0,1],[0,1,1,0],[0,1,1,1],[1,0,1,1]])

{0: 0, 1: 1, 2: 2, 3: 0}
{0: 0, 1: 1, 2: 1, 3: 0}
{0: 1, 1: 1, 2: 1, 3: 0}


1

In [46]:
# 305. Number of Islands II, m*n的grid上不断添加节点，更新当前岛屿个数
def numIslands2(m, n, positions):
    """
    :type m: int
    :type n: int
    :type positions: List[List[int]]
    :rtype: List[int]
    """
    grid = [[0 for _ in range(n)] for _ in range(m)]
    parents = dict()
    res = []
    count = 0
    def find(node):# get the parents of the node
        if parents[node] == node:
            return node
        parents[node] = find(parents[node])
        return parents[node]

    for i,j in positions:
        if grid[i][j]:
            res.append(count)
            continue
        grid[i][j]=1
        p = set()# it's neighbors
        for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            x,y=i+dx,j+dy
            if 0<=x<m and 0<=y<n and grid[x][y]:
                p.add(find((x,y)))
        if len(p)==0:# a new point
            parents[(i,j)] = (i,j)
            count+=1
            res.append(count)
        else:
            p = list(p)
            if len(p)==1:# neighbor to one island
                parents[(i,j)] = p[0]
                res.append(count)
            else:# neighbor to several islands
                parents[(i,j)] = p[0]
                for x,y in p[1:]:
                    parents[(x,y)] = p[0]
                    count-=1
                res.append(count)
    return res

def numIslands2(m, n, positions):
    parent, rank = {}, {}
    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]
    def union(x, y):
        x, y = find(x), find(y)
        if x == y:
            return 0
        if rank[x] < rank[y]:
            x, y = y, x
        parent[y] = x
        rank[x] += rank[x] == rank[y]
        return 1
    counts, count = [], 0
    for i, j in positions:
        x = parent[x] = i, j
        rank[x] = 0
        count += 1
        for y in (i+1, j), (i-1, j), (i, j+1), (i, j-1):
            if y in parent:
                count -= union(x, y)
        counts.append(count)
    return counts
numIslands2(8,6,[[0,5],[5,4],[5,2],[7,3],[6,0],[5,3],[5,1],[1,3],[4,3],[2,3],[3,5],[3,2],[3,0],[2,4],[0,1]])

[1, 2, 3, 4, 5, 4, 4, 5, 5, 5, 6, 7, 8, 8, 9]

In [15]:
# 684. Redundant Connection, 一个树里多了一条边，以至于得到了无向图，问多余的边
# 当一条边的两个顶点位于同一个图的时候说明重复了
def findRedundantConnection(edges):
    """
    :type edges: List[List[int]]
    :rtype: List[int]
    """
    parents = dict()
    def find(x):
        if parents[x] == x:
            return x
        parents[x] = find(parents[x])
        return parents[x]
        
    for x,y in edges:
        if x not in parents and y not in parents:#都不在已有的
            parents[x] = x
            parents[y] = x
        elif x in parents and y in parents:#都在已有的
            if find(x)==find(y):
                return [x,y]
            else:
                parents[find(x)] = find(y)#union的时候一定要调用find，union的是根节点
        else:
            if x not in parents:
                parents[x] = find(y)
            else:
                parents[y] = find(x)
findRedundantConnection([[16,25],[7,9],[3,24],[10,20],[15,24],[2,8],[19,21],[2,15],[13,20],[5,21],[7,11],[6,23],[7,16],[1,8],[17,20],[4,19],[11,22],[5,11],[1,16],[14,20],[1,4],[22,23],[12,20],[15,18],[12,16]])

[1, 4]

In [14]:
# 721. Accounts Merge，给定名字和账户，返回一个名字与其对应的所有账户
def accountsMerge(accounts):
    """
    :type accounts: List[List[str]]
    :rtype: List[List[str]]
    """
    parent = dict()
    rank = dict()
    names = dict()

    def find(i):
        if parent[i] == i:
            return i
        parent[i] = find(parent[i])
        return parent[i]
    
    def union(i,j):
        i = find(i)
        j = find(j)
        if rank[i] < rank[j]:#q rank更大
            parent[i] = j
        elif rank[i] > rank[j]:
            parent[j] = i
        else:
            parent[j] = i#p成了root，rank+1
            rank[i] += 1

    for account in accounts:
        name = account[0]
        first = account[1]
        if first not in parent:#对所有账户进行并入
            parent[first] = first
            rank[first] = 0
        for email in account[2:]:
            if email not in parent:
                parent[email] = find(account[1])
                rank[email] = 0
            else:
                union(email,account[1])
        if find(account[1]) not in names:
            names[find(account[1])] = name
                
    for i in parent:#考虑到有的账户没有并入干净
        find(i)

    emails = dict()
    newnames = dict()
    for key,item in parent.items():
        if item not in emails:
            emails[item] = []
        emails[item].append(key)
        if key in names and item not in newnames:
            newnames[item] = names[key]
    res = []
    for key,item in emails.items():
        res.append([newnames[key]]+sorted(item))
    return res

accountsMerge([["David","David0@m.co","David1@m.co"],["David","David3@m.co","David4@m.co"],["David","David4@m.co","David5@m.co"],["David","David2@m.co","David3@m.co"],["David","David1@m.co","David2@m.co"]])

[['David',
  'David0@m.co',
  'David1@m.co',
  'David2@m.co',
  'David3@m.co',
  'David4@m.co',
  'David5@m.co']]

In [25]:
# 261. Graph Valid Tree,给定一堆edges，判断是否只有一棵无环树
# 323. Number of Connected Components in an Undirected Graph，一堆edges判断有多少个group
def countComponents( n, edges):
    """
    :type n: int
    :type edges: List[List[int]]
    :rtype: int
    """
    def find(node):
        if parent[node] == node:
            return node
        parent[node] = find(parent[node])
        return parent[node]

    def union(i,j):
        i = find(i)
        j = find(j)
        if rank[i] < rank[j]:
            parent[i] = j
        elif rank[i] > rank[j]:
            parent[j] = i
        else:
            parent[i] = j
            rank[j]+=1

    parent = {i:i for i in range(n)}
    rank = {i:0 for i in range(n)}
    for i,j in edges:
        union(i,j)
        
    return len({find(x) for x in parent})#考虑到有的node没有并入干净
countComponents(4,[[0,1],[2,3],[1,2]])

In [33]:
# 323. Number of Connected Components in an Undirected Graph，一堆edges判断有多少个group
# 建立邻接矩阵，对每一个节点进行DFS遍历
def countComponents(n, edges):
    """
    :type n: int
    :type edges: List[List[int]]
    :rtype: int
    """
    visited = [False for _ in range(n)]
    graph = {i:set() for i in range(n)}
    for i,j in edges:
        graph[i].add(j)
        graph[j].add(i)
    print(graph)

    def dfs(i):
        if visited[i]:
            return
        visited[i] = True
        for j in graph[i]:
            dfs(j)
    res = 0  
    for i in range(n):
        if not visited[i]:
            res+=1
            dfs(i)
    return res
countComponents(4,[[0,1],[2,3],[1,2]])

In [1]:
# 1202. Smallest String With Swaps，字符串s，pairs中给定的index可以任意调换，求最小字符串
# 将pair归类到一个个group，group内面sort
def smallestStringWithSwaps(s, pairs):
    """
    :type s: str
    :type pairs: List[List[int]]
    :rtype: str
    """
    parent = [i for i in range(len(s))]
    rank = [0 for i in range(len(s))]

    def find(node):
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(i,j):
        p = find(i)
        q = find(j)
        if p==q:
            return
        if rank[p]<rank[q]:
            parent[p]=q
        elif rank[p]>rank[q]:
            parent[q] = p
        else:
            rank[p] += 1
            parent[q] = p
        return

    for i,j in pairs:
        union(i,j)
    parent = [find(i) for i in range(len(s))]

    dic = dict()
    for i in range(len(s)):
        if parent[i] not in dic:
            dic[parent[i]] = [[],[]]
        dic[parent[i]][0].append(i)
        dic[parent[i]][1].append(s[i])

    s = ["" for _ in range(len(s))]
    for key in dic:
        dic[key][1].sort()
        for i in range(len(dic[key][0])):
            s[dic[key][0][i]]=dic[key][1][i]
    return "".join(s)
smallestStringWithSwaps(s = "dcab", pairs = [[0,3],[1,2]])

'bacd'

In [2]:
# 1101. The Earliest Moment When Everyone Become Friends, 用groups来判断圈子的个数
def earliestAcq(logs, N):
    uf = {x: x for x in range(N)}
    groups = [N]

    def merge(x, y):
        x, y = find(x), find(y)
        if x != y:
            groups[0] -= 1
            uf[x] = y

    def find(x):
        if uf[x] != x:
            uf[x] = find(uf[x])
        return uf[x]

    for t, x, y in sorted(logs):
        merge(x, y)
        if groups[0] == 1:
            return t
    return -1
earliestAcq(logs = [[20190101,0,1],[20190104,3,4],[20190107,2,3],[20190211,1,5],[20190224,2,4],[20190301,0,3],[20190312,1,2],[20190322,4,5]], N = 6)

20190301

In [13]:
# 765. Couples Holding Hands，交换人，使得couple相邻

# 环形，数出每个圈的大小，长度为n的圈子交换次数为n-1
# greedy思想，每次交换一个人，要更新信息

def minSwapsCouples(row):
    """
    :type row: List[int]
    :rtype: int
    """
    length = len(row)
    visited = [False for _ in range(len(row))]
    res = 0
    pairs = dict()
    for i in range(length//2):
        pairs[row[i*2]] = row[2*i+1]
        pairs[row[2*i+1]] = row[2*i]

    def dfs(val,target):
        cur = pairs[val]
        visited[cur] = True
        step = 0
        while cur!=target:
            newtarget = cur+1 if cur%2==0 else cur-1
            visited[newtarget] = True
            cur = pairs[newtarget]
            visited[cur] = True
            step+=1
        return step

    res = 0
    for i in range(length//2):
        if not visited[row[2*i]]:
            target = row[2*i]+1 if row[2*i]%2==0 else row[2*i]-1
            visited[row[2*i]] = True
            res+=dfs(row[2*i],target)
    return res
    
def minSwapsCouples(row):
    """
    :type row: List[int]
    :rtype: int
    """
    graph = dict()
    for idx,pos in enumerate(row):
        graph[pos] = idx

    res = 0
    for i in range(0,len(row),2):
        cur = row[i]
        if cur%2==0:
            target = cur+1
        else:
            target = cur-1
        if row[i+1]!=target:
            pos = graph[target]
            row[i+1],row[pos] = row[pos],row[i+1]
            graph[row[i+1]] = i+1
            graph[row[pos]] = pos
            res+=1
    return res
minSwapsCouples([0,2,4,6,7,1,3,5])

3

In [1]:
# 1258. Synonymous Sentences,同义词替换，按照顺序输出结果
def generateSentences(synonyms, text):
    """
    :type synonyms: List[List[str]]
    :type text: str
    :rtype: List[str]
    """
    parent = dict()

    def find(node):
        if node not in parent:
            parent[node] = node
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(s1,s2):
        p,q = find(s1),find(s2)
        if p!=q:
            parent[q] = p

    for s1,s2 in synonyms:
        union(s1,s2)

    parent = {i:find(i) for i in parent}

    words = text.split(" ")
    dic = dict()
    for word in words:
        if word in parent:
            dic[word] = [i for i in parent if parent[i]==parent[word]]

    res = []
    def dfs(index,path,res):
        for i in range(index,len(words)):
            if words[i] in dic:
                for word in sorted(dic[words[i]]):
                    if word==words[i]:
                        dfs(i+1,path+words[index:i]+[word],res)
                    else:
                        dfs(i+1,path+words[index:i]+[word],res)
                return

        res.append(" ".join(path+words[index:]))   

    dfs(0,[],res)

    return res
generateSentences(synonyms = [["happy","joy"],["sad","sorrow"],["joy","cheerful"]],text = "I am happy today but was sad yesterday")

In [7]:
# 959. Regions Cut By Slashes,将每个格子分成4份，代表着不同形态的划分能产生的，可能划分方式，然后对不同小格子之间做联合
def regionsBySlashes(grid):
    f = {}
    def find(x):
        f.setdefault(x, x)
        if x != f[x]:
            f[x] = find(f[x])
        return f[x]
    def union(x, y):
        f[find(x)] = find(y)

    for i in range(len(grid)):
        for j in range(len(grid)):
            if i:#如果
                union((i - 1, j, 2), (i, j, 0))
            if j:
                union((i, j - 1, 1), (i, j, 3))
            if grid[i][j] != "/":
                union((i, j, 0), (i, j, 1))
                union((i, j, 2), (i, j, 3))
            if grid[i][j] != "\\":
                union((i, j, 3), (i, j, 0))
                union((i, j, 1), (i, j, 2))
    return len(set(map(find, f)))
regionsBySlashes(["\\/","/\\"])

4

In [8]:
# 947. Most Stones Removed with Same Row or Column,只要同行或者同列存在石头，就可以删除，问最多删除多少个
# 相当于同行同列的可以绑定在一起，也可以间接绑定在一起，相当于按照行列来分组，正数或者0为行，负数为列
def removeStones(stones):
    """
    :type stones: List[List[int]]
    :rtype: int
    """
    parent = dict()

    def find(i):
        if i not in parent:
            parent[i] = i
        if parent[i]!=i:
            parent[i] = find(parent[i])
        return parent[i]

    def union(i,j):
        parent[find(j)] = find(i)

    for i,j in stones:
        union(i,-j-1)

    return len(stones) - len(set(map(find,parent)))
removeStones(stones = [[0,0],[0,1],[1,0],[1,2],[2,1],[2,2]])

5

In [42]:
# 924. Minimize Malware Spread,病毒蔓延，去掉哪个病毒影响最大
def minMalwareSpread(graph, initial):
    """
    :type graph: List[List[int]]
    :type initial: List[int]
    :rtype: int
    """
    #每个group的大小，已经拥有的病毒数量
    N = len(graph)
    parent = {i:i for i in range(N)}

    def find(i):
        if parent[i]!=i:
            parent[i] = find(parent[i])
        return parent[i]

    def union(i,j):
        parent[find(j)] = find(i)

    for i in range(N):
        for j in range(i+1,N):
            if graph[i][j]:
                union(i,j)
    numbers = dict()
    for i in parent:#对于每个重要节点，其group的大小
        numbers[find(i)] = numbers.get(find(i),0)+1

    counts = dict()#病毒里所有可能的父节点与对应的病毒号
    for index in initial:#父节点，已经对应的
        counts[find(index)] = counts.get(find(index),[])+[index]

    initial.sort()
    maximum,res = 0, initial[0]#长度，以及对应的initial里的序列号
    for i in counts:#找到长度为1的病毒，并找出size最大的group
        if len(counts[i])==1:#对于长度为1的节点
            if numbers[i]>maximum:
                maximum,res = numbers[i],counts[i][0]
#     area = collections.Counter(find(i) for i in range(n))
#     malware = collections.Counter(find(i) for i in initial)
#     return min(initial, key=lambda i: [(malware[find(i)] == 1) * -area[find(i)], i])
    return res

def minMalwareSpread(graph, initial):
    def dfs(node,vis):#对于节点node，不断遍历并更新同组元素
        for v in  range(len(graph[node])):
            if graph[node][v] == 1 and v not in vis:
                vis.add(v)
                dfs(v,vis)

    s = set(initial)
    t_vis = set()
    del_node, subgraph_len = min(initial), 0
    for node in range(len(graph)):
        if node not in t_vis:#如果之前遍历过同组元素，那么没必要继续遍历一次
            vis = set([node])
            dfs(node,vis)
            # caculate the number of infected node in the subgraph
            infect = vis & s
            if len(infect) == 1:
                # more number of nodes or smaller index
                if len(vis) > subgraph_len or (len(vis) == subgraph_len and list(infect)[0] < del_node):
                    del_node,subgraph_len = list(infect)[0],len(vis)
            t_vis |= vis#将该组的添加到visited中
    return del_node
minMalwareSpread([[1,0,0,0,1,0,0,0],[0,1,1,0,0,1,0,0],[0,1,1,0,1,0,0,0],[0,0,0,1,1,0,0,0],[1,0,1,1,1,0,0,1],[0,1,0,0,0,1,0,0],[0,0,0,0,0,0,1,1],[0,0,0,0,1,0,1,1]],[7,2])

2

In [242]:
# 928. Minimize Malware Spread II
def minMalwareSpread(graph, initial):
    """
    :type graph: List[List[int]]
    :type initial: List[int]
    :rtype: int
    """
    def dfs(i,cur,delete):
        for j in range(len(graph)):
            if graph[i][j] and j not in cur and j!=delete:
                cur.add(j)
                dfs(j,cur,delete)

    virus = set(initial)
    visited = set()
    maximum,index = 0,float('inf')
    for i in initial:#对于病毒
        if i not in visited:#如果没有被访问过
            cur = {i}#组的大小
            dfs(i,cur,None)
            common = cur&virus#组内被感染的
            visited|=cur
            if len(common)==1:#组内只有一个节点被感染，组的大小
                if len(cur)>maximum or len(cur)==maximum and list(common)[0]<index:
                    maximum,index = len(cur),list(common)[0]
            else:#如果有多个节点被感染，那么对于每个都试试
                length = len(cur)
                for node in common:
                    curvirus = common - {node}
                    cur = set()
                    for curnode in curvirus:
                        if curnode not in cur:
                            cur.add(curnode)
                            dfs(curnode,cur,node)#对于病毒节点，进行遍历
                    if length-len(cur)>maximum or (length-len(cur)==maximum and node<index):
                        maximum,index = length-len(cur),node
    return index

import collections
def minMalwareSpread(graph, initial):#对于每个感染源，统计其被感染的节点的分布，如果被感染的节点只能被它一个感染，那么就纳入统计
    n = len(graph)
    d = collections.defaultdict(list)#每个未被感染节点是否能被直接感染
    # 对每个初始感染节点依次bfs，因为节点总数不超过300，所以不会超时
    for init in initial:#对于initial里的每个节点
        vis = set(initial)
        Q = collections.deque([init])
        while Q:
            infect = Q.popleft()
            for node in range(len(graph[infect])):#对于该节点下的其他节点
                if graph[infect][node] == 0: 
                    continue
                if node in vis: #如果本来就被感染了，那不考虑，排除其他节点的影响
                    continue
                vis.add(node)
                d[node].append(init)
                Q.append(node)
    # 统计出现最多次的感染节点
    res = [0] * n
    for key in d:
        if len(d[key]) == 1:#这个节点只能被感染一次，记录一下这个感染源头
            res[d[key][0]] += 1
    if max(res) == 0: return min(initial)
    return res.index(max(res))

import itertools
def minMalwareSpread(graph, initial):
    N = len(graph)
    clean = list(set(range(N)) - set(initial))#目前没被感染的
    parents, size = list(range(N)), [1] * N#这样可以计算根节点的size

    def find(a):
        while parents[a] != a:
            parents[a] = parents[parents[a]]
            a = parents[a]
        return a

    def union(a, b):
        p, q = find(a), find(b)
        if p != q:
            if size[p] > size[q]:#如果p size更大，那么就把p作为根节点，更新size
                parents[q] = p
                size[p] += size[q]
            else:
                parents[p] = q
                size[q] += size[p]

    for u, v in itertools.combinations(clean, 2):
        if graph[u][v]: #将没有被感染的关系确定在一起
            union(u, v)

    infect_node, infected_by, max, res = collections.defaultdict(set), collections.Counter(), -1, min(initial)

    for u in initial:#对于每个被感染的节点
        for v in clean:#对于没有被感染的节点，如果他们有连通，说明与其连通的点都会被感染
            if graph[u][v]:
                infect_node[u].add(find(v))
        for i in infect_node[u]:#每个感染源与其可以被感染到的父节点
            infected_by[i] += 1#对于每个可以被感染的父节点， 他们的感染源数量

    for i, nodes in infect_node.items():#对于每个感染源与其父节点
        count = 0#count为移除以后直接导致的没有被感染的数量
        for node in nodes:#如果被感染的父节点感染源只有一个，那么感染源就是来自这个，为直接被影响的节点
            if infected_by[node] == 1: 
                count += size[node]
        if count > max or count == max and i < res:
            max, res = count, i
    return res
minMalwareSpread([[1,1,1,0],[1,1,0,0],[1,0,1,0],[0,0,0,1]],[3,2])

2

In [190]:
# 952. Largest Component Size by Common Factor，一组数分成几个group，内部互不互质，最大group size
# 不互质的数相互union，最终最大的group size，但是这样双循环会超时
# 对每个数进行质因数分解，分解以后对每个质数进行统计，统计每个质数拥有的合数，然后对于这些合数，第一个合数与后面的合数进行union
# 质因数分解的方法只需要可能的质数进行到这个数的sqrt即可，不要走到这个数本身，否则会超时
def largestComponentSize(A):
    """
    :type A: List[int]
    :rtype: int
    """
    parent = {i:i for i in A}
    rank = {i:0 for i in A}
    def find(i):
        if parent[i] != i:
            parent[i] = find(parent[i])
        return parent[i]

    def union(i,j):
        p,q = find(i),find(j)
        if p!=q:
            if rank[p]>rank[q]:
                parent[q] = p
            elif rank[q] > rank[p]:
                parent[p] = q
            else:
                parent[q] = p
                rank[p]+=1

    def primeFactors(n):  # Prime factor decomposition，质因数分解的方法
        cur = set()
        while n % 2 == 0: 
            cur.add(2)
            n //= 2
        for i in range(3, int(n**0.5)+1, 2): 
            while n % i== 0: 
                cur.add(i) 
                n //= i 
        if n > 2: 
            cur.add(n)
        return cur

#     primes = [True for _ in range(max(A)+1)]
#     primes[0] = False
#     primes[1] = False
#     for i in range(1,len(primes)):
#         if primes[i]:
#             for j in range(i**2,len(primes),i):
#                 primes[j] = False
#     primes = [i for i,val in enumerate(primes) if val]
#     def primeFactors(num):
#         cur = set()
#         j = 0
#         while j<len(primes) and int(num**0.5)>=primes[j]:
#             if num%primes[j]==0:
#                 cur.add(primes[j])
#                 while num%primes[j]==0:
#                     num = num//primes[j]
#             j = j+1
#         if num!=1:
#             cur.add(num)
#         return cur

    arr = []
    for i in A:
        arr.append(primeFactors(i))
    decompositions = collections.defaultdict(list)   
    for i in range(len(A)):
        for t in arr[i]:
            decompositions[t].append(A[i])

    for key,val in decompositions.items():#同一个质数倍数union
        val.sort()
        for t in val[1:]:
            if find(val[0])!=find(t):
                union(val[0],t)
                             
#     primeToIndex = {} 
#     for a in A:
#         prime = primeFactors(a)
# #         print(prime)
#         for p in prime:#对于数中的质数
#             if p in primeToIndex:#如果质数在
#                 union(a, primeToIndex[p])
#             primeToIndex[p] = a
    return max(collections.Counter(find(i) for i in A).values())
nums = [84,171,548,709,455,967,328,779,79,16,168,675,276,473,697,26,731,668,634,607]
largestComponentSize(nums)

16

In [1]:
# 990. Satisfiability of Equality Equations，判断等式/不等式是否成立
# 把等式拿出来，相等的在一个group里，不等的留在最后一个个判断，如果是一个group，那么返回False
def equationsPossible(equations):
    """
    :type equations: List[str]
    :rtype: bool
    """
    parent = dict()

    def find(node):
        if node not in parent:
            parent[node] = node
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(i,j):
        parent[find(j)] = find(i)

    unequal = []
    for equation in equations:
        x,y = equation[0],equation[-1]
        if equation[1] == "=":
            union(x,y)
        else:
            unequal.append((x,y))

    for x,y in unequal:
        if x==y or (x in parent and y in parent and find(x)==find(y)):#可能出现a!=a，并且a之前没出现过的情况
            return False
    return True

In [3]:
# 1562. Find Latest Group of Size M
def findLatestStep(arr, m):
    """
    :type arr: List[int]
    :type m: int
    :rtype: int
    """
    if m==len(arr):
        return m
    def find(node):
        if node not in parent:
            parent[node] = node
            return node
        if parent[node]!=node:
            parent[node] = find(parent[node])
        return parent[node]

    parent = dict()
    lengths = dict()
    counts = dict()
    record = []
    for index,i in enumerate(arr):
        if i!=1 and i!=len(arr) and i+1 in parent and i-1 in parent:
            t1 = lengths[find(i+1)]
            t2 = lengths[find(i-1)]
            lengths.pop(find(i+1))
            parent[i] = find(i-1)
            parent[find(i+1)] = find(i-1)
            t = t1+t2+1
            counts[t1]-=1
            counts[t2]-=1
        elif i+1 in parent:
            t = lengths[find(i+1)]
            parent[i] = find(i+1)
            counts[t]-=1
            t = t+1
        elif i-1 in parent:
            t = lengths[find(i-1)]
            parent[i] = find(parent[i-1])
            counts[t]-=1
            t = t+1
        else:
            parent[i] = i
            t = 1
        lengths[find(i)] = t
        counts[t]=counts.get(t,0)+1
        if m in counts and counts[m]!=0:
            record.append(index+1)

    return record[-1] if record else -1   


def findLatestStep(arr, m):
    length = [0] * (len(arr) + 2)
    count = [0] * (len(arr) + 1)
    res = -1
    for i, a in enumerate(arr):
        left, right = length[a - 1], length[a + 1]
        # 左边的长度，右边的长度
        length[a] = length[a - left] = length[a + right] = left + right + 1
        count[left] -= 1
        count[right] -= 1
        count[length[a]] += 1
        if count[m]:
            res = i + 1
    return res
findLatestStep(arr = [3,5,1,2,4], m = 1)

4

In [20]:
# 1632. Rank Transform of a Matrix
import collections
def matrixRankTransform(A):
    n, m = len(A), len(A[0])
    rank = [0] * (m + n)
    d = collections.defaultdict(list)
    for i in range(n):
        for j in range(m):
            d[A[i][j]].append([i, j])
    print(d)

    def find(i):
        if p[i] != i:
            p[i] = find(p[i])
        return p[i]

    for a in sorted(d):
        p = [i for i in range(m+n)]
        rank2 = rank[:]
        for i, j in d[a]:
            i, j = find(i), find(j + n)#第i行，第j列
            p[i] = j
            rank2[j] = max(rank2[i], rank2[j])
        for i, j in d[a]:
            rank[i] = rank[j + n] = A[i][j] = rank2[find(i)] + 1
    return A

matrixRankTransform([[20,-21,14],[-19,4,19],[22,-47,24],[-19,4,19]])

defaultdict(<class 'list'>, {20: [[0, 0]], -21: [[0, 1]], 14: [[0, 2]], -19: [[1, 0], [3, 0]], 4: [[1, 1], [3, 1]], 19: [[1, 2], [3, 2]], 22: [[2, 0]], -47: [[2, 1]], 24: [[2, 2]]})


[[4, 2, 3], [1, 3, 4], [5, 1, 6], [1, 3, 4]]